In [14]:
# Load data
import numpy as np
from sklearn.model_selection import train_test_split
from Inner_Speech_Dataset.Python_Processing.Data_extractions import  Extract_data_from_subject

# Load all data for subject
def load_subject(subject_nr):
    datatype = "EEG"
    root_dir = "../dataset"

    data, description = Extract_data_from_subject(root_dir, subject_nr, datatype)
    return data, description

# Extract labels from the description
def extract_labels(desc):
    return desc[:,1]


# Test when extracting only the action interval
def extract_action_interval(data):
    return data[:,:,254:890]

def split_data(data, labels):
    trainv_data, test_data, trainv_labels, test_labels = train_test_split(data, labels, test_size = 0.1, random_state = 1, shuffle = True, stratify = labels)# 10% test data
    train_data, val_data, train_labels, val_labels = train_test_split(trainv_data, trainv_labels, test_size = 0.12, random_state = 1, shuffle = True, stratify = trainv_labels) # Approx 10% val data

    return train_data, val_data, test_data, train_labels, val_labels, test_labels

######### Check correct distr of split ################
def split_info(train_data, val_data, test_data, train_labels, val_labels, test_labels):
    t = [0,0,0,0]
    v = [0,0,0,0]
    te = [0,0,0,0]

    for l in train_labels:
        t[l] +=1

    for l in val_labels:
        v[l] +=1

    for l in test_labels:
        te[l] +=1

    print(t,"\n", v, "\n", te)

    print(t[0]/len(train_data)*100, "% ", t[1]/len(train_data)*100, "% ",t[2]/len(train_data)*100, "% ",t[3]/len(train_data)*100, "% ")
    print(v[0]/len(val_data)*100, "% ", v[1]/len(val_data)*100, "% ",v[2]/len(val_data)*100, "% ",v[3]/len(val_data)*100, "% ")
    print(te[0]/len(test_data)*100, "% ", te[1]/len(test_data)*100, "% ",te[2]/len(test_data)*100, "% ",te[3]/len(test_data)*100, "% ")


# Load, extract and split data
#data, description = load_subject(1)
#labels = extract_labels(description)
#new_data = extract_action_interval(data)
#print(new_data.shape)


#train_data, val_data, test_data, train_labels, val_labels, test_labels = split_data(data, labels)


#split_info(train_data, val_data, test_data, train_labels, val_labels, test_labels)




[99, 99, 99, 99] 
 [13, 13, 14, 14] 
 [13, 13, 12, 12]
25.0 %  25.0 %  25.0 %  25.0 % 
24.074074074074073 %  24.074074074074073 %  25.925925925925924 %  25.925925925925924 % 
26.0 %  26.0 %  24.0 %  24.0 % 
